# Introduction to DataFrames.jl

Loosely based on [this
tutorial](https://juliaai.github.io/DataScienceTutorials.jl/data/dataframe/)
from Data Science Tutorials.

DataFrames.jl **cheatsheets**:
[english](https://www.ahsmart.com/assets/pages/data-wrangling-with-data-frames-jl-cheat-sheet/DataFramesCheatSheet_v1.x_rev1.pdf),
[chinese](https://www.ahsmart.com/assets/pages/data-wrangling-with-data-frames-jl-cheat-sheet/DataFramesCheatSheet_v1.x_rev1_zh.pdf)

## Setup

Inspect Julia version:

In [1]:
VERSION

v"1.6.3"

The following instantiates a package environment.

The package environment has been created using **Julia 1.6** and may not
instantiate properly for other Julia versions.

In [2]:
using Pkg
Pkg.activate("env")
Pkg.instantiate()

  Activating environment at `~/GoogleDrive/Julia/HelloJulia/notebooks/02_dataframes/env/Project.toml`


## Loading some demonstration data

We'll be using [OpenML](https://www.openml.org/home) to grab datasets.

In [3]:
using OpenML
using DataFrames

table = OpenML.load(42638); # Titanic data set
typeof(table)

Tables.DictColumnTable

This is not a `DataFrame`. However, like
[many](https://github.com/JuliaData/Tables.jl/blob/main/INTEGRATIONS.md)
other tabular data containers in Julia, it can be converted to one in the
obvious way:

In [4]:
df = DataFrame(table);

A `DataFrame` is essentially just a wrapper around a number of
vectors with names, conceptualized as a table with the vectors as
columns.

Lets' look the first few rows (observations) of `df`:

In [5]:
first(df, 4)

,pclass,sex,age,sibsp,fare,cabin,embarked,survived
,Cat…,Cat…,Float64,Float64,Float64,Cat…?,Cat…?,Cat…
1,3,male,22.0,1.0,7.25,missing,S,0
2,1,female,38.0,1.0,71.2833,C85,C,1
3,3,female,26.0,0.0,7.925,missing,S,1
4,1,female,35.0,1.0,53.1,C123,S,1


Here's a summary of stats for each column:

In [6]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,pclass,,1,,3,0,"CategoricalValue{String, UInt32}"
2,sex,,female,,male,0,"CategoricalValue{String, UInt32}"
3,age,29.7589,0.42,30.0,80.0,0,Float64
4,sibsp,0.523008,0.0,0.0,8.0,0,Float64
5,fare,32.2042,0.0,14.4542,512.329,0,Float64
6,cabin,,E31,,C148,687,"Union{Missing, CategoricalValue{String, UInt32}}"
7,embarked,,C,,S,2,"Union{Missing, CategoricalValue{String, UInt32}}"
8,survived,,0,,1,0,"CategoricalValue{String, UInt32}"


To get just the column names, use `names(df)`.

## Selecting a single entry:

In [7]:
number = df[5, :age]   # or df[5, 4]

35.0

## Selecting a single row:

We use the wild-card `:`

In [8]:
row = df[5, :]

,pclass,sex,age,sibsp,fare,cabin,embarked,survived
,Cat…,Cat…,Float64,Float64,Float64,Cat…?,Cat…?,Cat…
5,3,male,35.0,0.0,8.05,missing,S,0


In [9]:
number = row.age

35.0

## Selecting multiple rows

By index:

In [10]:
small_df = df[3:7, :]

,pclass,sex,age,sibsp,fare,cabin,embarked,survived
,Cat…,Cat…,Float64,Float64,Float64,Cat…?,Cat…?,Cat…
1,3,female,26.0,0.0,7.925,missing,S,1
2,1,female,35.0,1.0,53.1,C123,S,1
3,3,male,35.0,0.0,8.05,missing,S,0
4,3,male,30.0,0.0,8.4583,missing,Q,0
5,1,male,54.0,0.0,51.8625,E46,S,0


By applying some criterion:

In [11]:
df2 = filter(df) do row
    row.age > 60 && row.survived == "1"
end

,pclass,sex,age,sibsp,fare,cabin,embarked,survived
,Cat…,Cat…,Float64,Float64,Float64,Cat…?,Cat…?,Cat…
1,1,female,63.0,1.0,77.9583,D7,S,1
2,3,female,63.0,0.0,9.5875,missing,S,1
3,2,male,62.0,0.0,10.5,missing,S,1
4,1,male,80.0,0.0,30.0,A23,S,1
5,1,female,62.0,0.0,80.0,B28,missing,1


(Alternatively, one can use `subset` or Boolean indexing; see the
[cheatsheet](https://www.ahsmart.com/assets/pages/data-wrangling-with-data-frames-jl-cheat-sheet/DataFramesCheatSheet_v1.x_rev1.pdf).)

## Selecting a single column

Here's how I get the column named "Age":

In [12]:
df2.age

5-element Vector{Float64}:
 63.0
 63.0
 62.0
 80.0
 62.0

Since this is just a vector, I can restrict the rows as usual:

In [13]:
df2.age[2:5]  # but same as df2[2:5, :Age]

4-element Vector{Float64}:
 63.0
 62.0
 80.0
 62.0

## Selecting multiple columns - `select`

The following wraps `df.age` as a single-column `DataFrame`:

In [14]:
select(df2, :age)

,age
,Float64
1,63.0
2,63.0
3,62.0
4,80.0
5,62.0


... which generalizes to multiple columns:

In [15]:
select(df2, [:age, :sex])

,age,sex
,Float64,Cat…
1,63.0,female
2,63.0,female
3,62.0,male
4,80.0,male
5,62.0,female


In [16]:
select(df2, Not(:age))

,pclass,sex,sibsp,fare,cabin,embarked,survived
,Cat…,Cat…,Float64,Float64,Cat…?,Cat…?,Cat…
1,1,female,1.0,77.9583,D7,S,1
2,3,female,0.0,9.5875,missing,S,1
3,2,male,0.0,10.5,missing,S,1
4,1,male,0.0,30.0,A23,S,1
5,1,female,0.0,80.0,B28,missing,1


In [17]:
select(df2, Between(:sex, :fare))

,sex,age,sibsp,fare
,Cat…,Float64,Float64,Float64
1,female,63.0,1.0,77.9583
2,female,63.0,0.0,9.5875
3,male,62.0,0.0,10.5
4,male,80.0,0.0,30.0
5,female,62.0,0.0,80.0


## Copies or views of the data?

So far, all the selection mechanisms discussed so far return a new
`DataFrame` object, with these exceptions:

* selecting a single row, as in `df[3,:]` - this returns a `DataFrameRow`.
* selecting a single column as a vector, as in `df.age` - this returns a vector

These exceptions are *views* of the original `DataFrame`, in the
sense that no data is copied and mutating the view mutates the
original `DataFrame`:

In [18]:
v = df.age;
v[1] = 1000
df[1, :age]

1000.0

Mutating the output of all the other selection mechanisms will not
touch the original `DataFrame`, as data is copied in those cases.

Use `select!` to make "in-place" column selections which mutate the
original `DataFrame`. For example, the following permanently drops a
column from `df`:

In [19]:
select!(df, Not(:cabin));
first(df, 2)

,pclass,sex,age,sibsp,fare,embarked,survived
,Cat…,Cat…,Float64,Float64,Float64,Cat…?,Cat…
1,3,male,1000.0,1.0,7.25,S,0
2,1,female,38.0,1.0,71.2833,C,1


In addition to row and column views of a `DataFrame` we can construct
larger views, of type `SubDataFrame`, which share most of the
behviour of an ordinary `DataFrame`, except they only *point* to
data in the parent `DataFrame`, rather than duplicate it:

In [20]:
df_copy = df[1:3, [:fare, :age]]       # this is copy
df_view = @view df[1:3, [:fare, :age]] # this is a view

,fare,age
,Float64,Float64
1,7.25,1000.0
2,71.2833,38.0
3,7.925,26.0


In [21]:
df_view[1, :age] = 4000
df[1, :age]

4000.0

In [22]:
df_copy[1, :age]

1000.0

## Describing the data

To broaden the summary provided by `describe(df)` we can pass a
number of symbols to indicate the statistics we are after:

In [23]:
describe(df, :min, :max, :mean, :median, :std)

,variable,min,max,mean,median,std
,Symbol,Any,Any,Union…,Union…,Union…
1,pclass,1,3,,,
2,sex,female,male,,,
3,age,0.42,4000.0,34.2235,30.0,133.642
4,sibsp,0.0,8.0,0.523008,0.0,1.10274
5,fare,0.0,512.329,32.2042,14.4542,49.6934
6,embarked,C,S,,,
7,survived,0,1,,,


The following are all supported:
* `mean`, `std`, `min`, `max`, `median`, `first`, `last` are all fairly self explanatory
* `q25`, `q75` are respectively for the 25th and 75th percentile,
* `eltype`, `nunique`, `nmissing` can also be used

You can also pass your custom function with a pair `function =>
:name` for instance:

In [24]:
using Statistics # to get functions like `mean` and `std`
foo(v) = mean(abs.(v))
d = describe(df, :mean, :median, foo => :mae)
first(d, 3)

,variable,mean,median,mae
,Symbol,Union…,Union…,Union…
1,pclass,,,
2,sex,,,
3,age,34.2235,30.0,34.2235


Note that the object returned by `describe` is itself a `DataFrame`:

In [25]:
select(d, [:variable, :mean])

,variable,mean
,Symbol,Union…
1,pclass,
2,sex,
3,age,34.2235
4,sibsp,0.523008
5,fare,32.2042
6,embarked,
7,survived,


## Materializing as a matrix

To convert the content of the dataframe as one big matrix do this:
use `convert`:

In [26]:
mat = Matrix(df)
mat[1:3, 1:3]

3×3 Matrix{Any}:
 "3"  "male"    4000.0
 "1"  "female"    38.0
 "3"  "female"    26.0

## Adding columns

Adding a column to a dataframe is very easy:

In [27]:
df.weird = df.fare ./ df.age;
first(df, 3)

,pclass,sex,age,sibsp,fare,embarked,survived,weird
,Cat…,Cat…,Float64,Float64,Float64,Cat…?,Cat…,Float64
1,3,male,4000.0,1.0,7.25,S,0,0.0018125
2,1,female,38.0,1.0,71.2833,C,1,1.87588
3,3,female,26.0,0.0,7.925,S,1,0.304808


That's it! Remember also that you can drop columns or make
subselections with `select` and `select!`.

The remainder of this tutorial is an abbreviated version of a
[DataFrames.jl
tutorial](http://juliadata.github.io/DataFrames.jl/latest/man/split_apply_combine/).

In [28]:
iris = OpenML.load(61) |> DataFrame;
first(iris, 3)

,sepallength,sepalwidth,petallength,petalwidth,class
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa


## `groupby`

The `groupby` function allows to form "sub-dataframes" corresponding
to groups of rows.  This can be very convenient to run specific
analyses for specific groups without copying the data.

The basic usage is `groupby(df, cols)` where `cols` specifies one or
several columns to use for the grouping.

Consider a simple example: in `iris` there is a `Species` column
with 3 species:

In [29]:
unique(iris.class)

3-element Vector{String}:
 "Iris-setosa"
 "Iris-versicolor"
 "Iris-virginica"

We can form views for each of these:

In [30]:
gdf = groupby(iris, :class);

The `gdf` object now corresponds to **views** of the original
dataframe for each of the 3 species; the first species
is `"Iris-setosa"` with:

In [31]:
subdf_setosa = gdf[1];
typeof(subdf_setosa)

SubDataFrame{DataFrame, DataFrames.Index, Vector{Int64}}

In [32]:
describe(subdf_setosa, :min, :mean, :max)

,variable,min,mean,max
,Symbol,Any,Union…,Any
1,sepallength,4.3,5.006,5.8
2,sepalwidth,2.3,3.418,4.4
3,petallength,1.0,1.464,1.9
4,petalwidth,0.1,0.244,0.6
5,class,Iris-setosa,,Iris-setosa


Recall that this means modifying `subdf_setosa` also modifies its
parent `iris`.

Do `?groupby` for more information.

## `combine`

The `combine` function allows to derive a new dataframe out of
transformations of an existing one.  Here's an example taken from
the official doc (see `?combine`):

In [33]:
df = DataFrame(a=1:3, b=4:6)

,a,b
,Int64,Int64
1,1,4
2,2,5
3,3,6


In [34]:
combine(df, :a => sum, nrow)

,a_sum,nrow
,Int64,Int64
1,6,3


What happened here is that the derived DataFrame has two columns
obtained respectively by (1) computing the sum of the first column
and (2) applying the `nrow` function on the `df`.

The transformation can produce one or several values, and `combine` will
try to accomodate this with appropriate packing:

In [35]:
foo(v) = v[1:2]
combine(df, :a => maximum, :b => foo)

,a_maximum,b_foo
,Int64,Int64
1,3,4
2,3,5


Here the maximum value of `a` is copied twice so that the two
columns have the same number of rows.

In [36]:
bar(v) = v[end-1:end]
combine(df, :a => foo, :b => bar)

,a_foo,b_bar
,Int64,Int64
1,1,5
2,2,6


## `combine` with `groupby`

Combining `groupby` with `combine` is very useful.  For instance you
might want to compute statistics across groups for different
variables:

In [37]:
combine(groupby(iris, :class), :petallength => mean)

,class,petallength_mean
,Cat…,Float64
1,Iris-setosa,1.464
2,Iris-versicolor,4.26
3,Iris-virginica,5.552


Let's break this operatioin down:

1. The `groupby(iris, :class)` creates groups using the `:class` column (which has values `Iris-setosa`, `Iris-versicolor`, `Iris-virginica`)
2. The `combine` creates a derived dataframe by applying the `mean` function to the `:petallength` column
3. Since there are three groups, we get one column (mean of `petallength`) and three rows (one per group).


You can do this for several columns/statistics at the time and give
new column names to the results:

In [38]:
gdf = groupby(iris, :class)
combine(gdf, :petallength => mean => :MPL, :petallength => std => :SPL)

,class,MPL,SPL
,Cat…,Float64,Float64
1,Iris-setosa,1.464,0.173511
2,Iris-versicolor,4.26,0.469911
3,Iris-virginica,5.552,0.551895


So here we assign the names `:MPL` and `:SPL` to the derived
columns.  If you want to apply something on all columns apart from
the grouping one, using `names` and `Not` comes in handy:

In [39]:
combine(gdf, names(iris, Not(:class)) .=> std)

,class,sepallength_std,sepalwidth_std,petallength_std,petalwidth_std
,Cat…,Float64,Float64,Float64,Float64
1,Iris-setosa,0.35249,0.381024,0.173511,0.10721
2,Iris-versicolor,0.516171,0.313798,0.469911,0.197753
3,Iris-virginica,0.63588,0.322497,0.551895,0.27465


where

In [40]:
names(iris, Not(:class))

4-element Vector{String}:
 "sepallength"
 "sepalwidth"
 "petallength"
 "petalwidth"

and note the use of `.` in `.=>` to indicate that we broadcast the
function over each column.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*